In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
import numpy as np

In [2]:
# Necessary preprocessing
df = pd.read_csv('dataset/activators.csv', encoding = 'latin1')
df = df.drop(columns = ['Name'])
df.fillna(df.mean(), inplace=True)
df = df.rename(columns = {'AC 50': 'label'})
df['label'] = df['label'].apply(lambda x: (str(x))[:-2])
df['label'] = df['label'].apply(lambda x: x.strip())
df['label'] = df['label'].astype(float)
df.head()

,label,nAcid,ALogP,ALogp2,AMR,apol,naAromAtom,nAromBond,nAtom,nHeavyAtom,...,AMW,WTPT-1,WTPT-2,WTPT-3,WTPT-4,WTPT-5,WPATH,WPOL,XLogP,Zagreb
0,0.06,1,-0.5682,0.322851,28.7271,43.427895,12,12,36,21,...,7.808478,42.782836,2.037278,10.880506,7.366041,3.514465,964,34,3.720,112
1,0.02,0,-2.3297,5.427502,49.2289,44.286688,6,6,40,24,...,8.501986,48.543887,2.022662,24.466222,24.466222,0.000000,1339,42,-0.015,128
2,0.05,0,-1.8662,3.482702,29.9218,42.307102,12,12,36,22,...,8.391084,44.711369,2.032335,16.171905,16.171905,0.000000,1038,38,0.453,118
3,0.06,1,-1.0722,1.149613,42.5079,33.727516,0,0,29,17,...,8.002924,33.928275,1.995781,13.895974,7.420051,6.475923,470,29,1.631,88
4,0.02,0,-0.1366,0.018660,10.4302,34.671930,12,12,27,17,...,8.224744,35.290030,2.075884,5.747701,5.747701,0.000000,500,26,4.859,90


In [3]:
features = list(df.columns)[1:]
print (features[:10])

['nAcid', 'ALogP', 'ALogp2', 'AMR', 'apol', 'naAromAtom', 'nAromBond', 'nAtom', 'nHeavyAtom', 'nH']


In [4]:
# Convert AC50 to categories
label_map = ["less than 1", "more than/equal to 1 and less than 30", "more than/equal to 30 and less than 60", "more than/equal to 60 and less than 100", "more than/equal to 100"]
for idx in df.index:
    if df['label'][idx] < 1:
        df['label'][idx] = 0
    elif df['label'][idx] >= 1 and df['label'][idx] < 30:
        df['label'][idx] = 1
    elif df['label'][idx] >=30 and df['label'][idx] < 60:
        df['label'][idx] = 2
    elif df['label'][idx] >= 60 and df['label'][idx] < 100:
        df['label'][idx] = 3
    elif df['label'][idx] >= 100:
        df['label'][idx] = 4
        
df['label'] = df['label'].astype(int)
df.head()

C:\Users\Yuvraj\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\Yuvraj\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\Yuvraj\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Yuvraj\Ana

,label,nAcid,ALogP,ALogp2,AMR,apol,naAromAtom,nAromBond,nAtom,nHeavyAtom,...,AMW,WTPT-1,WTPT-2,WTPT-3,WTPT-4,WTPT-5,WPATH,WPOL,XLogP,Zagreb
0,0,1,-0.5682,0.322851,28.7271,43.427895,12,12,36,21,...,7.808478,42.782836,2.037278,10.880506,7.366041,3.514465,964,34,3.720,112
1,0,0,-2.3297,5.427502,49.2289,44.286688,6,6,40,24,...,8.501986,48.543887,2.022662,24.466222,24.466222,0.000000,1339,42,-0.015,128
2,0,0,-1.8662,3.482702,29.9218,42.307102,12,12,36,22,...,8.391084,44.711369,2.032335,16.171905,16.171905,0.000000,1038,38,0.453,118
3,0,1,-1.0722,1.149613,42.5079,33.727516,0,0,29,17,...,8.002924,33.928275,1.995781,13.895974,7.420051,6.475923,470,29,1.631,88
4,0,0,-0.1366,0.018660,10.4302,34.671930,12,12,27,17,...,8.224744,35.290030,2.075884,5.747701,5.747701,0.000000,500,26,4.859,90


In [5]:
# Necessary preprocessing
x = df.loc[:, features].values
x = StandardScaler().fit_transform(x)
pca = PCA(n_components = 10)   
principalComponents = pca.fit_transform(x)
pdf = pd.DataFrame(data = principalComponents)
df = pd.concat([pdf, df[['label']]], axis = 1)
features = list(df.columns)[:-1]

In [16]:
# # Uncomment this cell to re-train the model

# # Splitting data into test and train datasets
# train, test = train_test_split(df, test_size = 0.2)
# X_train, y_train, X_test, y_test = train[features], list(train['label']), test[features], list(test['label'])

# # Fit the model
# rf = RandomForestClassifier(max_depth = 5, max_features= 2, n_estimators = 15)
# rf = rf.fit(X_train, y_train)
# print ("Train accuracy : " + str(rf.score(X_train, y_train)))
# print ("Test accuracy : " + str(rf.score(X_test, y_test)))

Train accuracy : 1.0
Test accuracy : 0.875


In [7]:
# Load trained model
import joblib

filename = 'models/activators_model.sav'
rf = joblib.load(filename)

rf.get_params()

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 5,
 'max_features': 2,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 15,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [8]:
# Check model accuracy
X, y = df[features], list(df['label'])
print ("Overall model accuracy on entire (test + train) dataset : " + str(rf.score(X, y)))

Overall model accuracy on entire (test + train) dataset : 0.9736842105263158


In [9]:
# Predict AC 50 for new molecules
tdf = pd.read_csv('test/activators_test.csv', encoding = 'latin1')
tdf = tdf.drop(columns = ['Name'])
tdf.fillna(0, inplace=True)
tx = tdf.values
tx = StandardScaler().fit_transform(tx)
tprincipalComponents = pca.transform(tx)
tpdf = pd.DataFrame(data = tprincipalComponents)
x_test = tpdf.values

preds = np.array(rf.predict(x_test))
for i in range(preds.size):
    print (str(i+1) + ". AC50 is " + label_map[preds[i]] + " um")

1. AC50 is less than 1 um
2. AC50 is less than 1 um
3. AC50 is less than 1 um
